In [1]:
spark

In [2]:
infile = 'file:///home/cloudera/2.kkbox_churn/data01/from_raw_transactions-v1/0.with_corrected_dates'
df0 = spark.read.format('csv').option('header','true').load(infile)
df0.printSchema()

root
 |-- msno: string (nullable = true)
 |-- pay_method: string (nullable = true)
 |-- plan_days: string (nullable = true)
 |-- list_price: string (nullable = true)
 |-- actual_paid: string (nullable = true)
 |-- auto_renew: string (nullable = true)
 |-- is_cancel: string (nullable = true)
 |-- trans_date: string (nullable = true)
 |-- expire_date: string (nullable = true)
 |-- corr_exp_date: string (nullable = true)
 |-- corr_start_date: string (nullable = true)



In [3]:
infile = '/home/cloudera/2.kkbox_churn/data01/from_raw_transactions-v1/1.problematic_members'
df1 = spark.read.format('csv').option('header','true').load(infile)
df1.printSchema()

root
 |-- msno: string (nullable = true)



## perform anti-join 

In [5]:
df2 = df0.join(df1, df0['msno']==df1['msno'], 'left_anti')
df2.printSchema()

root
 |-- msno: string (nullable = true)
 |-- pay_method: string (nullable = true)
 |-- plan_days: string (nullable = true)
 |-- list_price: string (nullable = true)
 |-- actual_paid: string (nullable = true)
 |-- auto_renew: string (nullable = true)
 |-- is_cancel: string (nullable = true)
 |-- trans_date: string (nullable = true)
 |-- expire_date: string (nullable = true)
 |-- corr_exp_date: string (nullable = true)
 |-- corr_start_date: string (nullable = true)



In [6]:
outfile = 'file:///home/cloudera/2.kkbox_churn/data01/from_raw_transactions-v1/2.problematic_members_excluded'
df2.write.format('csv').option('header','true').save(outfile)

## remove records with `corr_exp_date` = null add `tid`

In [3]:
infile = 'file:///home/cloudera/2.kkbox_churn/data01/from_raw_transactions-v1/2.problematic_members_excluded'
df0 = spark.read.format('csv').option('header','true').load(infile)
df0.printSchema()

root
 |-- msno: string (nullable = true)
 |-- pay_method: string (nullable = true)
 |-- plan_days: string (nullable = true)
 |-- list_price: string (nullable = true)
 |-- actual_paid: string (nullable = true)
 |-- auto_renew: string (nullable = true)
 |-- is_cancel: string (nullable = true)
 |-- trans_date: string (nullable = true)
 |-- expire_date: string (nullable = true)
 |-- corr_exp_date: string (nullable = true)
 |-- corr_start_date: string (nullable = true)



In [13]:
from pyspark.sql.functions import col
df1 = df0.where(~col('corr_exp_date').isNull())

In [14]:
from pyspark.sql.functions import monotonically_increasing_id
df2 = df1.select(monotonically_increasing_id().alias('tid'), '*')
df2.printSchema()

root
 |-- tid: long (nullable = false)
 |-- msno: string (nullable = true)
 |-- pay_method: string (nullable = true)
 |-- plan_days: string (nullable = true)
 |-- list_price: string (nullable = true)
 |-- actual_paid: string (nullable = true)
 |-- auto_renew: string (nullable = true)
 |-- is_cancel: string (nullable = true)
 |-- trans_date: string (nullable = true)
 |-- expire_date: string (nullable = true)
 |-- corr_exp_date: string (nullable = true)
 |-- corr_start_date: string (nullable = true)



In [15]:
df2.limit(5).toPandas()

,tid,msno,pay_method,plan_days,list_price,actual_paid,auto_renew,is_cancel,trans_date,expire_date,corr_exp_date,corr_start_date
0,0,5mJ0DPOvKzdvIM1LD/72I/LShb+HVXz0Kl/XuS5Oht8=,41,30,149,149,true,false,2015-11-10,2015-12-10,2015-12-10,2015-11-11
1,1,YxIAl59LEP2qpKbDq3UfW6I9KMJg5DV/4LKei2vaKHg=,36,30,180,180,true,false,2016-08-31,2016-10-02,2016-10-02,2016-09-03
2,2,beDQknpc/HYBUkqNdJGRholzO4rYP7urzjJoIr9xtQw=,41,30,149,0,true,true,2016-01-31,2016-01-31,2016-01-31,None
3,3,/7RhQbsl148r+Je9VZA5bP+HUXo7TJJWW+EzR6BKuLs=,40,30,149,149,true,false,2016-10-22,2016-11-22,2016-11-22,2016-10-24
4,4,Wb30CbaCSyk8eTKC5Qfi9sLn84U6G4RStTBBvMMp2Og=,36,30,150,150,false,false,2016-03-17,2016-04-16,2016-04-16,2016-03-18


In [16]:
# output
outfile = 'file:///home/cloudera/2.kkbox_churn/data01/from_raw_transactions-v1/2a.problematic_members_excluded'
df2.write.format('csv').option('header','true').save(outfile)

## split data set into divisions
Each member belongs to exactly 1 division.

In [ ]:
infile = '/home/cloudera/2.kkbox_churn/data01/from_raw_transactions-v1/2a.problematic_members_excluded'
df0 = spark.read.format('csv').option('header','true').load(infile)
df0.printSchema()

In [ ]:
from pyspark.sql.functions import col
df = df0.where(col('msno') >= 'A').where(col('msno') < 'O')

dir0 = '/home/cloudera/2.kkbox_churn/data01/from_raw_transactions-v1/2a.problematic_members_excluded/'
outfile = dir0 + 'msno_A-N'
df.write.format('csv').option('header','true').save(outfile)

In [ ]:
df = df0.where(col('msno') >= 'O').where(col('msno') < '[')

outfile = dir0 + 'msno_O-Z'
df.write.format('csv').option('header','true').save(outfile)

In [ ]:
df = df0.where(col('msno') >= 'a').where(col('msno') < 'o')

outfile = dir0 + 'msno_a-n'
df.write.format('csv').option('header','true').save(outfile)

In [ ]:
df = df0.where(col('msno') >= 'o').where(col('msno') < '{')

outfile = dir0 + 'msno_o-z'
df.write.format('csv').option('header','true').save(outfile)

In [ ]:
df = df0.where((col('msno')<'A') | (col('msno')>='[')) \
        .where((col('msno')<'a') | (col('msno')>='{'))

outfile = dir0 + 'msno_non-letter'
df.write.format('csv').option('header','true').save(outfile)